# Calculate urban areas from gridded population data

In [57]:
import sys, os, importlib, json
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import shape, Polygon

#Import raster helpers
sys.path.append("../../../gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

#Import GOST urban functions
sys.path.append("../../")
import src.UrbanRaster as urban
import src.urban_helper as helper



In [58]:
iso3 = "HTI"
out_folder = "/home/wb411133/temp/%s_URBAN" % iso3
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
pop_file = os.path.join(out_folder, "WP_POP.tif")
urban_file = os.path.join(out_folder, "URBAN.tif")
    
global_bounds = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
global_pop = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"

inG = gpd.read_file(global_bounds)
selG = inG.loc[inG['ISO3'] == iso3]
inPop_raster = rasterio.open(global_pop)

if selG.crs != inPop_raster.crs:
    selG = selG.to_crs(inPop_raster.crs)
    
if not os.path.exists(pop_file):
    rMisc.clipRaster(inPop_raster, selG, pop_file)

In [59]:
urban_calc = urban.urbanGriddedPop(pop_file)
urban_extents = urban_calc.calculateUrban(raster=urban_file)

# Calculate Rural Extents
This is not commonly used, but there are times when having rural extents can be useful. This will generate a rural raster and a rural vector dataset

In [60]:
urban_raster = rasterio.open(urban_file)
urban_d = urban_raster.read()
rural_d = np.absolute(urban_d - 1)

In [61]:
rural_file = os.path.join(out_folder, "RURAL.tif")
with rMisc.create_rasterio_inmemory(urban_raster.profile, rural_d) as ruralR:
    rMisc.clipRaster(ruralR, selG, rural_file)

In [62]:
inR = rasterio.open(rural_file)
inD = inR.read()
all_features = []
# create vector results
for cShape, value in features.shapes(inD, transform=urban_raster.transform, connectivity=8):
    if value > 0:
        all_features.append([value, shape(geojson.loads(json.dumps(cShape)))])

In [63]:
geoms = gpd.GeoDataFrame(pd.DataFrame(all_features, columns=['VALUE','geometry']), geometry='geometry', crs=urban_raster.crs)
geoms['area'] = geoms['geometry'].apply(lambda x: x.area)
#Limit rural areas to those larger than 2 pixels
geoms = geoms.loc[geoms['area'] >= (urban_raster.res[0] * urban_raster.res[1]) * 2]

In [64]:
geoms.to_file(rural_file.replace(".tif", ".geojson"), driver="GeoJSON")